In [1]:
#################################
#           battery EV          #
#################################

__version__ = '0.3'

import pandas as pd
import numpy as np
from datetime import timedelta


In [2]:
class price_profiles:
    def __init__(self,request_status):
        self.request_status = request_status
    
    def getProfiles(self):
        import_price1 = pd.read_csv('data/octopus_imprt_price_southern.csv', index_col=0,parse_dates=True,header = 0)
        export_price1 = pd.read_csv('data/octopus_export_price_southern.csv', index_col=0,parse_dates=True,header = 0)

        start_time1 = "2019-1-31 00:00:00"
        end_time1 = "2020-1-30 22:30:00"
        
        start_time2 = "2018-1-31 00:00:00"
        end_time2 = "2019-1-30 22:30:00"
        
        import_price2 =import_price1.loc[start_time1:end_time1]
        export_price2 =export_price1.loc[start_time2:end_time2]
        
        import_price3 = import_price2.between_time('09:00:00','16:30:00')
        export_price3 = export_price2.between_time('09:00:00','16:30:00')
        
        for column in import_price3 [['Import Rate (p/kWh)']]:
            import_price = import_price3[column] 
            import_price_value = import_price.values
            
        for column in export_price3 [['Outgoing Agile Rate (p/kWh)']]:
            export_price = export_price3[column]
            export_price_value = export_price.values 
            
        return import_price_value, export_price_value 

In [3]:
class building_profiles:
    def __init__(self,request_status):
        self.request_status = request_status
        
        
    def getProfiles(self):
        excess_profile1 = pd.read_csv('data/Export_vector.csv', index_col=0, parse_dates=True)  # W/kWp
        excess_profile2 = excess_profile1.resample('0.5H').mean()
        excess_profile3 = excess_profile2.append(pd.DataFrame({excess_profile2.columns[0]: np.nan},
                                              index=[(excess_profile2.index[-1] +
                                                      timedelta(minutes=30))]))
        excess_profile4 = excess_profile3.interpolate()
        excess_profile5 = excess_profile4.between_time('09:00:00','16:30:00')
        excess_vector = excess_profile5.values


        import_profile1 = pd.read_csv('data/Import_vector.csv', index_col=0,parse_dates=True)  # W/kWp
        import_profile2 = import_profile1.resample('0.5H').mean()
        import_profile3 = import_profile2.append(pd.DataFrame({import_profile2.columns[0]: np.nan},
                                              index=[(import_profile2.index[-1] +timedelta(minutes=30))]))
        import_profile4 = import_profile3.interpolate()
        import_profile5 = import_profile4.between_time('09:00:00','16:30:00')
        import_vector = import_profile5.values
     
        return import_vector,excess_vector
        

In [4]:
class control_class:
    def __init__(self,request_status):
        self.request_status = request_status
        
    def signalGen(self,import_price_value,export_price_value, max_buy_price, min_sell_price,import_vector,excess_vector):     
        T = len(import_price_value) 
        signal = np.zeros((T, 1))
        
        if self.request_status == 0:  
            for time,price in enumerate(import_price_value):
                if price < max_buy_price:   # Charge/Buys when price < Maximum_Buy_Price
                    signal[time] = 1
                else:
                    signal[time] = 0  # Set all other cases to "do nothing" first
                    
            for time,price in enumerate(export_price_value):
                if price > min_sell_price :  # Discharge/Sells when price > Minimum_Sell_Price 
                    signal[time] = -1 # Corrects some cases to sell electricity 
                
        elif self.request_status == 1:       
            for time,excess_amount in enumerate(excess_vector):
                if excess_amount > 0:   # Charge the batteries for free when excess generation 
                    signal[time] = 1
                else:
                    signal[time] = 0  # Set all other cases to "do nothing" first
                    
            for time,import_amount in enumerate(import_vector):
                if import_amount > 0 :  # Discharge when building in need for more electricity  
                    signal[time] = -1 # Corrects some cases to discharge
                print(time,signal[time])
                
        return signal

In [5]:
class EV_Component:
    def __init__(self):
        self.dispatch_type = "EV Component"
        self.capacity = 0
        self.install_cost = 0
        self.depreciation_cost = 0
        self.lifetime = 15    # Assume all components have a lifespan of 15 years 

In [14]:
class EV_battery(EV_Component):
    def __init__(self, capacity, power, efficiency, dt, install_cost, signal):
        super().__init__()
        self.asset_type = 'EV Battery'
        self.capacity = capacity
        self.power = power * dt   # Convert kW to kWh
        self.eff = efficiency
        self.soc = np.ones(signal.size) * self.capacity
        self.install_cost = install_cost * 100  # p/kWh
        self.depreciation_cost = install_cost/self.lifetime
        self.signal = signal
        
    def getOutput(self, signal,request_status,import_vector,excess_vector,energy_reserved):
        T = len(signal)
        output = np.zeros((T, 1))   # zero array of T rows, 1 column
        
        if request_status == 0:
            for time, signal_status in enumerate(signal):
                if time % 16 == 0:           # Reset the soc every morning at 9 am  
                    soc = self.capacity - 5  # Capacity minus the energy used for commuting to work 
                else:
                    soc = self.soc[time - 1]           
                if signal[time] == -1 and self.soc[time-1]>energy_reserved:       # Signal indicating selling/ Discharging 
                                                                # Whilst also reserving enough for traveling home
                    output[time] = min(self.power, self.eff*(soc-energy_reserved))
                    self.soc[time] = soc - (1/self.eff)*output[time]
                elif signal[time] == 1:  # Signal indicating buying/ Charging 
                    output[time] = max(-self.power, - (1/self.eff) * (self.capacity - soc))
                    self.soc[time] = soc - self.eff * output[time]        
                else:  # do nothing
                    self.soc[time] = soc
            
        if request_status == 1:
            for time, signal_status in enumerate(signal):
                if time % 16 == 0:           # Reset the soc every morning at 9 am  
                    soc = self.capacity - 5  # Capacity minus the energy used for commuting to work 
                else:
                    soc = self.soc[time - 1]           
                if signal[time] == -1 and self.soc[time-1]>energy_reserved:       # Signal indicating discharging
                                                                # Whilst also reserving enough for traveling home
                    output[time] = min(self.power, self.eff*(soc-energy_reserved),self.eff*import_vector[time])
                    self.soc[time] = soc -(1/self.eff)*output[time]
                elif signal[time] == 1:  # Signal indicating Charging 
                    output[time] = max(-self.power, -(1/self.eff)*(self.capacity - soc),-(1/self.eff)*excess_vector[time])
                    self.soc[time] = soc - self.eff * output[time]        
                else:  # do nothing
                    self.soc[time] = soc
                print(time,output[time],self.soc[time])
        return output 
    
    def getGridCost(self, output, import_price_value, export_price_value,request_status,import_vector,excess_vector):
        cost_arr_size = len(output)
        cost_array = np.zeros((cost_arr_size, 1))   # zero array of T rows, 1 column
        
        for time, output_power in enumerate(output):
            if request_status == 0:
                if output[time] > 0:    # Positive output power, selling/discharging
                    cost_array[time] = - export_price_value[time]*output[time] #sell cost is negative
                elif output[time] < 0:  # Negative output power, buying/charging
                    cost_array[time] = - import_price_value[time]*output[time] #buy cost is positive           
                elif output[time] == 0:  # No power output, do nothing 
                    cost_array[time] = 0         
                #print(cost_array[time])
                
            if request_status == 1:
                if output[time] > 0:    # Positive output power, discharging to the buildings
                    # if building demand is not met, some electricity needs to be bought from grid
                    cost_array[time] = import_price_value[time]*(import_vector[time]-output[time]) #buy cost is positive
                elif output[time] < 0:  # Negative output power, charging from the excess of the builidngs 
                    # if building excess is too much, some electricity needs to be sold to grid
                    cost_array[time] = export_price_value[time]*(-excess_vector[time]-output[time]) #sell cost is negative         
                elif output[time] == 0:  # No power output, do nothing 
                    cost_array[time] = 0         
                print(time,cost_array[time]/100) 
                
                
                
        grid_cost = np.sum(cost_array) # in pence 
        grid_cost_GBP = grid_cost/100  # convert to pounds
        print('Grid cost: £ %3.3f' % grid_cost_GBP)
        return grid_cost_GBP
    

In [15]:
class V2G_Charger(EV_Component): 
    def __init__(self, install_cost = 0):
        super().__init__()
        self.asset_type = 'EV Charger'
        self.install_cost = install_cost * 100  # p/kWh

In [16]:
#######################
##   Main Programme  ##
#######################
dt = 60/60 # One-hour intervals 
request_status = 1 # Buildings require electricity status

max_buy_price = 4
min_sell_price = 7
energy_reserved = 5

In [17]:
# Price 
price_profiles1 = price_profiles(request_status)
import_price_value, export_price_value = price_profiles1.getProfiles()

In [18]:
# Building Profiles 
building_profiles1 = building_profiles(request_status)
import_vector1,excess_vector1 = building_profiles1.getProfiles()

In [19]:
# Control Signals
signal = control_class(request_status)
signal1 = signal.signalGen(import_price_value, export_price_value, max_buy_price, min_sell_price,import_vector1,excess_vector1)

0 [-1.]
1 [-1.]
2 [-1.]
3 [-1.]
4 [-1.]
5 [-1.]
6 [-1.]
7 [-1.]
8 [-1.]
9 [-1.]
10 [-1.]
11 [-1.]
12 [-1.]
13 [-1.]
14 [-1.]
15 [-1.]
16 [0.]
17 [0.]
18 [0.]
19 [0.]
20 [0.]
21 [0.]
22 [0.]
23 [0.]
24 [0.]
25 [0.]
26 [-1.]
27 [-1.]
28 [-1.]
29 [0.]
30 [0.]
31 [0.]
32 [0.]
33 [0.]
34 [1.]
35 [1.]
36 [1.]
37 [1.]
38 [1.]
39 [1.]
40 [1.]
41 [1.]
42 [1.]
43 [1.]
44 [1.]
45 [1.]
46 [1.]
47 [0.]
48 [1.]
49 [1.]
50 [1.]
51 [1.]
52 [1.]
53 [1.]
54 [1.]
55 [1.]
56 [1.]
57 [1.]
58 [1.]
59 [1.]
60 [1.]
61 [1.]
62 [1.]
63 [0.]
64 [1.]
65 [1.]
66 [1.]
67 [1.]
68 [1.]
69 [1.]
70 [1.]
71 [1.]
72 [1.]
73 [1.]
74 [1.]
75 [1.]
76 [1.]
77 [1.]
78 [1.]
79 [1.]
80 [1.]
81 [1.]
82 [1.]
83 [1.]
84 [1.]
85 [1.]
86 [1.]
87 [1.]
88 [1.]
89 [1.]
90 [1.]
91 [1.]
92 [1.]
93 [1.]
94 [1.]
95 [1.]
96 [1.]
97 [1.]
98 [1.]
99 [1.]
100 [1.]
101 [1.]
102 [1.]
103 [1.]
104 [1.]
105 [1.]
106 [1.]
107 [1.]
108 [1.]
109 [1.]
110 [1.]
111 [1.]
112 [1.]
113 [1.]
114 [1.]
115 [1.]
116 [1.]
117 [1.]
118 [1.]
119 [1.]
120 [1.]
12

1149 [1.]
1150 [1.]
1151 [0.]
1152 [1.]
1153 [1.]
1154 [1.]
1155 [1.]
1156 [1.]
1157 [1.]
1158 [1.]
1159 [1.]
1160 [1.]
1161 [1.]
1162 [1.]
1163 [1.]
1164 [1.]
1165 [1.]
1166 [1.]
1167 [0.]
1168 [1.]
1169 [1.]
1170 [1.]
1171 [1.]
1172 [1.]
1173 [1.]
1174 [1.]
1175 [1.]
1176 [1.]
1177 [1.]
1178 [1.]
1179 [1.]
1180 [1.]
1181 [1.]
1182 [1.]
1183 [0.]
1184 [1.]
1185 [1.]
1186 [1.]
1187 [1.]
1188 [1.]
1189 [1.]
1190 [1.]
1191 [1.]
1192 [1.]
1193 [1.]
1194 [1.]
1195 [1.]
1196 [1.]
1197 [1.]
1198 [1.]
1199 [0.]
1200 [1.]
1201 [1.]
1202 [1.]
1203 [1.]
1204 [1.]
1205 [1.]
1206 [1.]
1207 [1.]
1208 [1.]
1209 [1.]
1210 [1.]
1211 [1.]
1212 [1.]
1213 [0.]
1214 [0.]
1215 [0.]
1216 [0.]
1217 [0.]
1218 [0.]
1219 [0.]
1220 [1.]
1221 [1.]
1222 [1.]
1223 [1.]
1224 [1.]
1225 [1.]
1226 [1.]
1227 [1.]
1228 [1.]
1229 [1.]
1230 [1.]
1231 [0.]
1232 [1.]
1233 [1.]
1234 [1.]
1235 [1.]
1236 [1.]
1237 [1.]
1238 [1.]
1239 [1.]
1240 [1.]
1241 [1.]
1242 [1.]
1243 [1.]
1244 [1.]
1245 [1.]
1246 [1.]
1247 [0.]
1248 [1.]


2149 [1.]
2150 [1.]
2151 [1.]
2152 [1.]
2153 [1.]
2154 [1.]
2155 [1.]
2156 [1.]
2157 [1.]
2158 [1.]
2159 [1.]
2160 [1.]
2161 [1.]
2162 [1.]
2163 [1.]
2164 [1.]
2165 [1.]
2166 [1.]
2167 [1.]
2168 [1.]
2169 [1.]
2170 [1.]
2171 [1.]
2172 [1.]
2173 [1.]
2174 [1.]
2175 [1.]
2176 [1.]
2177 [1.]
2178 [1.]
2179 [1.]
2180 [1.]
2181 [1.]
2182 [1.]
2183 [1.]
2184 [1.]
2185 [1.]
2186 [1.]
2187 [1.]
2188 [1.]
2189 [1.]
2190 [1.]
2191 [1.]
2192 [1.]
2193 [1.]
2194 [1.]
2195 [1.]
2196 [1.]
2197 [1.]
2198 [1.]
2199 [1.]
2200 [1.]
2201 [1.]
2202 [1.]
2203 [1.]
2204 [1.]
2205 [1.]
2206 [1.]
2207 [1.]
2208 [1.]
2209 [1.]
2210 [1.]
2211 [1.]
2212 [1.]
2213 [1.]
2214 [1.]
2215 [1.]
2216 [1.]
2217 [1.]
2218 [1.]
2219 [1.]
2220 [1.]
2221 [1.]
2222 [1.]
2223 [1.]
2224 [1.]
2225 [1.]
2226 [1.]
2227 [1.]
2228 [1.]
2229 [1.]
2230 [1.]
2231 [1.]
2232 [1.]
2233 [1.]
2234 [1.]
2235 [1.]
2236 [1.]
2237 [1.]
2238 [1.]
2239 [0.]
2240 [1.]
2241 [1.]
2242 [1.]
2243 [1.]
2244 [1.]
2245 [1.]
2246 [1.]
2247 [1.]
2248 [1.]


3270 [1.]
3271 [1.]
3272 [1.]
3273 [1.]
3274 [1.]
3275 [1.]
3276 [1.]
3277 [1.]
3278 [1.]
3279 [1.]
3280 [1.]
3281 [1.]
3282 [1.]
3283 [1.]
3284 [1.]
3285 [1.]
3286 [1.]
3287 [1.]
3288 [1.]
3289 [1.]
3290 [1.]
3291 [1.]
3292 [1.]
3293 [1.]
3294 [1.]
3295 [1.]
3296 [1.]
3297 [1.]
3298 [1.]
3299 [1.]
3300 [1.]
3301 [1.]
3302 [1.]
3303 [1.]
3304 [1.]
3305 [1.]
3306 [1.]
3307 [1.]
3308 [1.]
3309 [1.]
3310 [1.]
3311 [1.]
3312 [1.]
3313 [1.]
3314 [1.]
3315 [1.]
3316 [1.]
3317 [1.]
3318 [1.]
3319 [1.]
3320 [1.]
3321 [1.]
3322 [1.]
3323 [1.]
3324 [1.]
3325 [1.]
3326 [1.]
3327 [1.]
3328 [1.]
3329 [1.]
3330 [1.]
3331 [1.]
3332 [1.]
3333 [1.]
3334 [1.]
3335 [1.]
3336 [1.]
3337 [1.]
3338 [1.]
3339 [1.]
3340 [1.]
3341 [1.]
3342 [1.]
3343 [1.]
3344 [1.]
3345 [1.]
3346 [1.]
3347 [1.]
3348 [1.]
3349 [1.]
3350 [1.]
3351 [1.]
3352 [1.]
3353 [1.]
3354 [1.]
3355 [1.]
3356 [1.]
3357 [1.]
3358 [1.]
3359 [1.]
3360 [1.]
3361 [1.]
3362 [1.]
3363 [1.]
3364 [1.]
3365 [1.]
3366 [1.]
3367 [1.]
3368 [1.]
3369 [1.]


4397 [0.]
4398 [0.]
4399 [0.]
4400 [1.]
4401 [1.]
4402 [1.]
4403 [1.]
4404 [1.]
4405 [1.]
4406 [1.]
4407 [1.]
4408 [1.]
4409 [1.]
4410 [1.]
4411 [1.]
4412 [1.]
4413 [1.]
4414 [1.]
4415 [0.]
4416 [1.]
4417 [1.]
4418 [1.]
4419 [1.]
4420 [1.]
4421 [1.]
4422 [1.]
4423 [1.]
4424 [1.]
4425 [1.]
4426 [1.]
4427 [1.]
4428 [1.]
4429 [1.]
4430 [1.]
4431 [1.]
4432 [1.]
4433 [1.]
4434 [1.]
4435 [1.]
4436 [1.]
4437 [1.]
4438 [1.]
4439 [1.]
4440 [1.]
4441 [1.]
4442 [1.]
4443 [1.]
4444 [1.]
4445 [1.]
4446 [1.]
4447 [1.]
4448 [1.]
4449 [1.]
4450 [1.]
4451 [1.]
4452 [1.]
4453 [1.]
4454 [1.]
4455 [1.]
4456 [1.]
4457 [1.]
4458 [1.]
4459 [1.]
4460 [1.]
4461 [1.]
4462 [1.]
4463 [1.]
4464 [1.]
4465 [1.]
4466 [1.]
4467 [1.]
4468 [1.]
4469 [1.]
4470 [1.]
4471 [1.]
4472 [1.]
4473 [1.]
4474 [1.]
4475 [1.]
4476 [1.]
4477 [1.]
4478 [1.]
4479 [1.]
4480 [0.]
4481 [0.]
4482 [1.]
4483 [1.]
4484 [1.]
4485 [1.]
4486 [1.]
4487 [1.]
4488 [1.]
4489 [1.]
4490 [1.]
4491 [1.]
4492 [1.]
4493 [1.]
4494 [1.]
4495 [1.]
4496 [1.]


5396 [1.]
5397 [1.]
5398 [1.]
5399 [1.]
5400 [1.]
5401 [1.]
5402 [1.]
5403 [1.]
5404 [1.]
5405 [1.]
5406 [1.]
5407 [1.]
5408 [1.]
5409 [1.]
5410 [1.]
5411 [1.]
5412 [1.]
5413 [1.]
5414 [1.]
5415 [1.]
5416 [1.]
5417 [1.]
5418 [1.]
5419 [1.]
5420 [1.]
5421 [1.]
5422 [1.]
5423 [1.]
5424 [1.]
5425 [1.]
5426 [1.]
5427 [1.]
5428 [1.]
5429 [1.]
5430 [1.]
5431 [1.]
5432 [1.]
5433 [1.]
5434 [1.]
5435 [1.]
5436 [1.]
5437 [1.]
5438 [1.]
5439 [1.]
5440 [1.]
5441 [1.]
5442 [1.]
5443 [1.]
5444 [1.]
5445 [1.]
5446 [1.]
5447 [1.]
5448 [1.]
5449 [1.]
5450 [1.]
5451 [1.]
5452 [1.]
5453 [1.]
5454 [1.]
5455 [1.]
5456 [1.]
5457 [1.]
5458 [1.]
5459 [1.]
5460 [1.]
5461 [1.]
5462 [1.]
5463 [1.]
5464 [1.]
5465 [1.]
5466 [1.]
5467 [1.]
5468 [1.]
5469 [1.]
5470 [1.]
5471 [1.]
5472 [1.]
5473 [1.]
5474 [1.]
5475 [1.]
5476 [1.]
5477 [1.]
5478 [1.]
5479 [1.]
5480 [1.]
5481 [1.]
5482 [1.]
5483 [1.]
5484 [1.]
5485 [1.]
5486 [1.]
5487 [1.]
5488 [0.]
5489 [0.]
5490 [0.]
5491 [0.]
5492 [0.]
5493 [0.]
5494 [0.]
5495 [0.]


In [20]:
# Battery
battery_capacity = 60  # kWh
battery_power = 10  # kW
battery_eff = 0.8
battery_cost = 0 
battery_site1 = EV_battery(battery_capacity,battery_power,battery_eff, dt, battery_cost,signal1)
output1 = battery_site1.getOutput(signal1,request_status,import_vector1,excess_vector1,energy_reserved)

0 [10.] 42.5
1 [10.] 30.0
2 [10.] 17.5
3 [10.] 5.0
4 [0.] 5.0
5 [0.] 5.0
6 [0.] 5.0
7 [0.] 5.0
8 [0.] 5.0
9 [0.] 5.0
10 [0.] 5.0
11 [0.] 5.0
12 [0.] 5.0
13 [0.] 5.0
14 [0.] 5.0
15 [0.] 5.0
16 [0.] 55.0
17 [0.] 55.0
18 [0.] 55.0
19 [0.] 55.0
20 [0.] 55.0
21 [0.] 55.0
22 [0.] 55.0
23 [0.] 55.0
24 [0.] 55.0
25 [0.] 55.0
26 [10.] 42.5
27 [10.] 30.0
28 [10.] 17.5
29 [0.] 17.5
30 [0.] 17.5
31 [0.] 17.5
32 [0.] 55.0
33 [0.] 55.0
34 [-6.25] 60.0
35 [-0.] 60.0
36 [-0.] 60.0
37 [-0.] 60.0
38 [-0.] 60.0
39 [-0.] 60.0
40 [-0.] 60.0
41 [-0.] 60.0
42 [-0.] 60.0
43 [-0.] 60.0
44 [-0.] 60.0
45 [-0.] 60.0
46 [-0.] 60.0
47 [0.] 60.0
48 [-6.25] 60.0
49 [-0.] 60.0
50 [-0.] 60.0
51 [-0.] 60.0
52 [-0.] 60.0
53 [-0.] 60.0
54 [-0.] 60.0
55 [-0.] 60.0
56 [-0.] 60.0
57 [-0.] 60.0
58 [-0.] 60.0
59 [-0.] 60.0
60 [-0.] 60.0
61 [-0.] 60.0
62 [-0.] 60.0
63 [0.] 60.0
64 [-6.25] 60.0
65 [-0.] 60.0
66 [-0.] 60.0
67 [-0.] 60.0
68 [-0.] 60.0
69 [-0.] 60.0
70 [-0.] 60.0
71 [-0.] 60.0
72 [-0.] 60.0
73 [-0.] 60.0
74 [-0.] 6

534 [0.] 55.0
535 [0.] 55.0
536 [10.] 42.5
537 [10.] 30.0
538 [10.] 17.5
539 [0.] 17.5
540 [0.] 17.5
541 [0.] 17.5
542 [0.] 17.5
543 [0.] 17.5
544 [0.] 55.0
545 [0.] 55.0
546 [0.] 55.0
547 [0.] 55.0
548 [-6.25] 60.0
549 [-0.] 60.0
550 [-0.] 60.0
551 [-0.] 60.0
552 [-0.] 60.0
553 [-0.] 60.0
554 [-0.] 60.0
555 [-0.] 60.0
556 [-0.] 60.0
557 [-0.] 60.0
558 [-0.] 60.0
559 [0.] 60.0
560 [-6.25] 60.0
561 [-0.] 60.0
562 [-0.] 60.0
563 [-0.] 60.0
564 [-0.] 60.0
565 [-0.] 60.0
566 [-0.] 60.0
567 [-0.] 60.0
568 [-0.] 60.0
569 [-0.] 60.0
570 [-0.] 60.0
571 [-0.] 60.0
572 [-0.] 60.0
573 [-0.] 60.0
574 [-0.] 60.0
575 [-0.] 60.0
576 [-6.25] 60.0
577 [-0.] 60.0
578 [-0.] 60.0
579 [-0.] 60.0
580 [-0.] 60.0
581 [-0.] 60.0
582 [-0.] 60.0
583 [-0.] 60.0
584 [-0.] 60.0
585 [-0.] 60.0
586 [-0.] 60.0
587 [-0.] 60.0
588 [-0.] 60.0
589 [-0.] 60.0
590 [-0.] 60.0
591 [-0.] 60.0
592 [-6.25] 60.0
593 [-0.] 60.0
594 [-0.] 60.0
595 [-0.] 60.0
596 [-0.] 60.0
597 [-0.] 60.0
598 [-0.] 60.0
599 [-0.] 60.0
600 [-0.] 60.0

1119 [0.] 60.0
1120 [0.] 55.0
1121 [0.] 55.0
1122 [-6.25] 60.0
1123 [-0.] 60.0
1124 [-0.] 60.0
1125 [-0.] 60.0
1126 [-0.] 60.0
1127 [-0.] 60.0
1128 [-0.] 60.0
1129 [-0.] 60.0
1130 [-0.] 60.0
1131 [-0.] 60.0
1132 [-0.] 60.0
1133 [0.] 60.0
1134 [0.] 60.0
1135 [0.] 60.0
1136 [-6.25] 60.0
1137 [-0.] 60.0
1138 [-0.] 60.0
1139 [-0.] 60.0
1140 [-0.] 60.0
1141 [-0.] 60.0
1142 [-0.] 60.0
1143 [-0.] 60.0
1144 [-0.] 60.0
1145 [-0.] 60.0
1146 [-0.] 60.0
1147 [-0.] 60.0
1148 [-0.] 60.0
1149 [-0.] 60.0
1150 [-0.] 60.0
1151 [0.] 60.0
1152 [-6.25] 60.0
1153 [-0.] 60.0
1154 [-0.] 60.0
1155 [-0.] 60.0
1156 [-0.] 60.0
1157 [-0.] 60.0
1158 [-0.] 60.0
1159 [-0.] 60.0
1160 [-0.] 60.0
1161 [-0.] 60.0
1162 [-0.] 60.0
1163 [-0.] 60.0
1164 [-0.] 60.0
1165 [-0.] 60.0
1166 [-0.] 60.0
1167 [0.] 60.0
1168 [-6.25] 60.0
1169 [-0.] 60.0
1170 [-0.] 60.0
1171 [-0.] 60.0
1172 [-0.] 60.0
1173 [-0.] 60.0
1174 [-0.] 60.0
1175 [-0.] 60.0
1176 [-0.] 60.0
1177 [-0.] 60.0
1178 [-0.] 60.0
1179 [-0.] 60.0
1180 [-0.] 60.0
1181 [-0

1687 [-0.] 60.0
1688 [-0.] 60.0
1689 [-0.] 60.0
1690 [-0.] 60.0
1691 [-0.] 60.0
1692 [-0.] 60.0
1693 [-0.] 60.0
1694 [-0.] 60.0
1695 [-0.] 60.0
1696 [-6.25] 60.0
1697 [-0.] 60.0
1698 [-0.] 60.0
1699 [-0.] 60.0
1700 [-0.] 60.0
1701 [-0.] 60.0
1702 [-0.] 60.0
1703 [-0.] 60.0
1704 [-0.] 60.0
1705 [-0.] 60.0
1706 [-0.] 60.0
1707 [-0.] 60.0
1708 [-0.] 60.0
1709 [-0.] 60.0
1710 [-0.] 60.0
1711 [0.] 60.0
1712 [-6.25] 60.0
1713 [-0.] 60.0
1714 [-0.] 60.0
1715 [-0.] 60.0
1716 [-0.] 60.0
1717 [-0.] 60.0
1718 [-0.] 60.0
1719 [-0.] 60.0
1720 [-0.] 60.0
1721 [-0.] 60.0
1722 [-0.] 60.0
1723 [-0.] 60.0
1724 [-0.] 60.0
1725 [-0.] 60.0
1726 [-0.] 60.0
1727 [-0.] 60.0
1728 [-6.25] 60.0
1729 [-0.] 60.0
1730 [-0.] 60.0
1731 [-0.] 60.0
1732 [-0.] 60.0
1733 [-0.] 60.0
1734 [-0.] 60.0
1735 [-0.] 60.0
1736 [-0.] 60.0
1737 [-0.] 60.0
1738 [-0.] 60.0
1739 [-0.] 60.0
1740 [-0.] 60.0
1741 [-0.] 60.0
1742 [-0.] 60.0
1743 [-0.] 60.0
1744 [0.] 55.0
1745 [0.] 55.0
1746 [0.] 55.0
1747 [0.] 55.0
1748 [-6.25] 60.0
1749 

2199 [-0.] 60.0
2200 [-0.] 60.0
2201 [-0.] 60.0
2202 [-0.] 60.0
2203 [-0.] 60.0
2204 [-0.] 60.0
2205 [-0.] 60.0
2206 [-0.] 60.0
2207 [-0.] 60.0
2208 [-6.25] 60.0
2209 [-0.] 60.0
2210 [-0.] 60.0
2211 [-0.] 60.0
2212 [-0.] 60.0
2213 [-0.] 60.0
2214 [-0.] 60.0
2215 [-0.] 60.0
2216 [-0.] 60.0
2217 [-0.] 60.0
2218 [-0.] 60.0
2219 [-0.] 60.0
2220 [-0.] 60.0
2221 [-0.] 60.0
2222 [-0.] 60.0
2223 [-0.] 60.0
2224 [-6.25] 60.0
2225 [-0.] 60.0
2226 [-0.] 60.0
2227 [-0.] 60.0
2228 [-0.] 60.0
2229 [-0.] 60.0
2230 [-0.] 60.0
2231 [-0.] 60.0
2232 [-0.] 60.0
2233 [-0.] 60.0
2234 [-0.] 60.0
2235 [-0.] 60.0
2236 [-0.] 60.0
2237 [-0.] 60.0
2238 [-0.] 60.0
2239 [0.] 60.0
2240 [-6.25] 60.0
2241 [-0.] 60.0
2242 [-0.] 60.0
2243 [-0.] 60.0
2244 [-0.] 60.0
2245 [-0.] 60.0
2246 [-0.] 60.0
2247 [-0.] 60.0
2248 [-0.] 60.0
2249 [-0.] 60.0
2250 [-0.] 60.0
2251 [-0.] 60.0
2252 [-0.] 60.0
2253 [-0.] 60.0
2254 [-0.] 60.0
2255 [-0.] 60.0
2256 [-6.25] 60.0
2257 [-0.] 60.0
2258 [-0.] 60.0
2259 [-0.] 60.0
2260 [-0.] 60.0
2

2865 [-0.] 60.0
2866 [-0.] 60.0
2867 [-0.] 60.0
2868 [-0.] 60.0
2869 [-0.] 60.0
2870 [-0.] 60.0
2871 [-0.] 60.0
2872 [-0.] 60.0
2873 [-0.] 60.0
2874 [-0.] 60.0
2875 [-0.] 60.0
2876 [-0.] 60.0
2877 [-0.] 60.0
2878 [-0.] 60.0
2879 [-0.] 60.0
2880 [-6.25] 60.0
2881 [-0.] 60.0
2882 [-0.] 60.0
2883 [-0.] 60.0
2884 [-0.] 60.0
2885 [-0.] 60.0
2886 [-0.] 60.0
2887 [-0.] 60.0
2888 [-0.] 60.0
2889 [-0.] 60.0
2890 [-0.] 60.0
2891 [-0.] 60.0
2892 [-0.] 60.0
2893 [-0.] 60.0
2894 [-0.] 60.0
2895 [-0.] 60.0
2896 [4.456] 49.43
2897 [0.] 49.43
2898 [0.] 49.43
2899 [0.] 49.43
2900 [0.] 49.43
2901 [0.] 49.43
2902 [0.] 49.43
2903 [0.] 49.43
2904 [0.] 49.43
2905 [0.] 49.43
2906 [6.452] 41.365
2907 [10.] 28.865000000000002
2908 [6.452] 20.800000000000004
2909 [0.] 20.800000000000004
2910 [0.] 20.800000000000004
2911 [0.] 20.800000000000004
2912 [-6.25] 60.0
2913 [-0.] 60.0
2914 [-0.] 60.0
2915 [-0.] 60.0
2916 [-0.] 60.0
2917 [-0.] 60.0
2918 [-0.] 60.0
2919 [-0.] 60.0
2920 [-0.] 60.0
2921 [-0.] 60.0
2922 [-0

3417 [-0.] 60.0
3418 [8.074] 49.9075
3419 [10.] 37.4075
3420 [8.074] 27.314999999999998
3421 [0.] 27.314999999999998
3422 [0.] 27.314999999999998
3423 [0.] 27.314999999999998
3424 [-6.25] 60.0
3425 [-0.] 60.0
3426 [-0.] 60.0
3427 [-0.] 60.0
3428 [-0.] 60.0
3429 [-0.] 60.0
3430 [-0.] 60.0
3431 [-0.] 60.0
3432 [-0.] 60.0
3433 [-0.] 60.0
3434 [-0.] 60.0
3435 [-0.] 60.0
3436 [-0.] 60.0
3437 [-0.] 60.0
3438 [-0.] 60.0
3439 [0.] 60.0
3440 [-6.25] 60.0
3441 [-0.] 60.0
3442 [-0.] 60.0
3443 [-0.] 60.0
3444 [-0.] 60.0
3445 [-0.] 60.0
3446 [-0.] 60.0
3447 [-0.] 60.0
3448 [-0.] 60.0
3449 [-0.] 60.0
3450 [-0.] 60.0
3451 [-0.] 60.0
3452 [-0.] 60.0
3453 [-0.] 60.0
3454 [-0.] 60.0
3455 [-0.] 60.0
3456 [0.] 55.0
3457 [0.] 55.0
3458 [0.] 55.0
3459 [0.] 55.0
3460 [-6.25] 60.0
3461 [-0.] 60.0
3462 [-0.] 60.0
3463 [-0.] 60.0
3464 [-0.] 60.0
3465 [-0.] 60.0
3466 [-0.] 60.0
3467 [-0.] 60.0
3468 [-0.] 60.0
3469 [-0.] 60.0
3470 [-0.] 60.0
3471 [-0.] 60.0
3472 [-6.25] 60.0
3473 [-0.] 60.0
3474 [-0.] 60.0
3475 [

4031 [-0.] 60.0
4032 [-6.25] 60.0
4033 [-0.] 60.0
4034 [-0.] 60.0
4035 [-0.] 60.0
4036 [-0.] 60.0
4037 [-0.] 60.0
4038 [-0.] 60.0
4039 [-0.] 60.0
4040 [-0.] 60.0
4041 [-0.] 60.0
4042 [-0.] 60.0
4043 [-0.] 60.0
4044 [-0.] 60.0
4045 [-0.] 60.0
4046 [-0.] 60.0
4047 [0.] 60.0
4048 [-6.25] 60.0
4049 [-0.] 60.0
4050 [-0.] 60.0
4051 [-0.] 60.0
4052 [-0.] 60.0
4053 [-0.] 60.0
4054 [-0.] 60.0
4055 [-0.] 60.0
4056 [-0.] 60.0
4057 [-0.] 60.0
4058 [-0.] 60.0
4059 [-0.] 60.0
4060 [-0.] 60.0
4061 [0.] 60.0
4062 [0.] 60.0
4063 [0.] 60.0
4064 [0.] 55.0
4065 [0.] 55.0
4066 [10.] 42.5
4067 [10.] 30.0
4068 [10.] 17.5
4069 [10.] 5.0
4070 [0.] 5.0
4071 [0.] 5.0
4072 [0.] 5.0
4073 [0.] 5.0
4074 [0.] 5.0
4075 [0.] 5.0
4076 [0.] 5.0
4077 [0.] 5.0
4078 [0.] 5.0
4079 [0.] 5.0
4080 [-6.25] 60.0
4081 [-0.] 60.0
4082 [-0.] 60.0
4083 [-0.] 60.0
4084 [-0.] 60.0
4085 [-0.] 60.0
4086 [-0.] 60.0
4087 [-0.] 60.0
4088 [-0.] 60.0
4089 [-0.] 60.0
4090 [-0.] 60.0
4091 [-0.] 60.0
4092 [-0.] 60.0
4093 [-0.] 60.0
4094 [-0.] 60

4698 [-0.] 60.0
4699 [-0.] 60.0
4700 [-0.] 60.0
4701 [-0.] 60.0
4702 [-0.] 60.0
4703 [0.] 60.0
4704 [0.] 55.0
4705 [0.] 55.0
4706 [0.] 55.0
4707 [0.] 55.0
4708 [-6.25] 60.0
4709 [-0.] 60.0
4710 [-0.] 60.0
4711 [-0.] 60.0
4712 [-0.] 60.0
4713 [-0.] 60.0
4714 [-0.] 60.0
4715 [-0.] 60.0
4716 [-0.] 60.0
4717 [0.] 60.0
4718 [0.] 60.0
4719 [0.] 60.0
4720 [0.] 55.0
4721 [0.] 55.0
4722 [0.] 55.0
4723 [0.] 55.0
4724 [0.] 55.0
4725 [0.] 55.0
4726 [0.] 55.0
4727 [0.] 55.0
4728 [0.] 55.0
4729 [0.] 55.0
4730 [0.] 55.0
4731 [0.] 55.0
4732 [0.] 55.0
4733 [0.] 55.0
4734 [0.] 55.0
4735 [0.] 55.0
4736 [0.] 55.0
4737 [0.] 55.0
4738 [0.] 55.0
4739 [0.] 55.0
4740 [0.] 55.0
4741 [0.] 55.0
4742 [0.] 55.0
4743 [0.] 55.0
4744 [0.] 55.0
4745 [0.] 55.0
4746 [0.] 55.0
4747 [0.] 55.0
4748 [0.] 55.0
4749 [0.] 55.0
4750 [0.] 55.0
4751 [0.] 55.0
4752 [-6.25] 60.0
4753 [-0.] 60.0
4754 [-0.] 60.0
4755 [-0.] 60.0
4756 [-0.] 60.0
4757 [-0.] 60.0
4758 [-0.] 60.0
4759 [-0.] 60.0
4760 [-0.] 60.0
4761 [-0.] 60.0
4762 [0.422]

5368 [10.] 42.5
5369 [10.] 30.0
5370 [10.] 17.5
5371 [10.] 5.0
5372 [0.] 5.0
5373 [0.] 5.0
5374 [0.] 5.0
5375 [0.] 5.0
5376 [0.] 55.0
5377 [0.] 55.0
5378 [-6.25] 60.0
5379 [-0.] 60.0
5380 [-0.] 60.0
5381 [-0.] 60.0
5382 [-0.] 60.0
5383 [-0.] 60.0
5384 [-0.] 60.0
5385 [-0.] 60.0
5386 [-0.] 60.0
5387 [-0.] 60.0
5388 [-0.] 60.0
5389 [-0.] 60.0
5390 [-0.] 60.0
5391 [0.] 60.0
5392 [-6.25] 60.0
5393 [-0.] 60.0
5394 [-0.] 60.0
5395 [-0.] 60.0
5396 [-0.] 60.0
5397 [-0.] 60.0
5398 [-0.] 60.0
5399 [-0.] 60.0
5400 [-0.] 60.0
5401 [-0.] 60.0
5402 [-0.] 60.0
5403 [-0.] 60.0
5404 [-0.] 60.0
5405 [-0.] 60.0
5406 [-0.] 60.0
5407 [-0.] 60.0
5408 [-6.25] 60.0
5409 [-0.] 60.0
5410 [-0.] 60.0
5411 [-0.] 60.0
5412 [-0.] 60.0
5413 [-0.] 60.0
5414 [-0.] 60.0
5415 [-0.] 60.0
5416 [-0.] 60.0
5417 [-0.] 60.0
5418 [-0.] 60.0
5419 [-0.] 60.0
5420 [-0.] 60.0
5421 [-0.] 60.0
5422 [-0.] 60.0
5423 [-0.] 60.0
5424 [-6.25] 60.0
5425 [-0.] 60.0
5426 [-0.] 60.0
5427 [-0.] 60.0
5428 [-0.] 60.0
5429 [-0.] 60.0
5430 [-0.] 6

In [21]:
grid_cost =  battery_site1.getGridCost(output1, import_price_value, export_price_value, request_status, import_vector1,excess_vector1)

0 [7.03401615]
1 [6.00472215]
2 [5.2428243]
3 [4.62643046]
4 [0.]
5 [0.]
6 [0.]
7 [0.]
8 [0.]
9 [0.]
10 [0.]
11 [0.]
12 [0.]
13 [0.]
14 [0.]
15 [0.]
16 [0.]
17 [0.]
18 [0.]
19 [0.]
20 [0.]
21 [0.]
22 [0.]
23 [0.]
24 [0.]
25 [0.]
26 [1.36105961]
27 [3.86419478]
28 [1.2134997]
29 [0.]
30 [0.]
31 [0.]
32 [0.]
33 [0.]
34 [-6.78250125]
35 [0.]
36 [0.]
37 [0.]
38 [0.]
39 [0.]
40 [0.]
41 [0.]
42 [0.]
43 [0.]
44 [0.]
45 [0.]
46 [0.]
47 [0.]
48 [-22.9707605]
49 [0.]
50 [0.]
51 [0.]
52 [0.]
53 [0.]
54 [0.]
55 [0.]
56 [0.]
57 [0.]
58 [0.]
59 [0.]
60 [0.]
61 [0.]
62 [0.]
63 [0.]
64 [-11.63911186]
65 [0.]
66 [0.]
67 [0.]
68 [0.]
69 [0.]
70 [0.]
71 [0.]
72 [0.]
73 [0.]
74 [0.]
75 [0.]
76 [0.]
77 [0.]
78 [0.]
79 [0.]
80 [-61.3688075]
81 [0.]
82 [0.]
83 [0.]
84 [0.]
85 [0.]
86 [0.]
87 [0.]
88 [0.]
89 [0.]
90 [0.]
91 [0.]
92 [0.]
93 [0.]
94 [0.]
95 [0.]
96 [-51.627936]
97 [0.]
98 [0.]
99 [0.]
100 [0.]
101 [0.]
102 [0.]
103 [0.]
104 [0.]
105 [0.]
106 [0.]
107 [0.]
108 [0.]
109 [0.]
110 [0.]
111 [0.]
112

852 [0.]
853 [0.]
854 [0.]
855 [0.]
856 [0.]
857 [0.]
858 [0.]
859 [0.]
860 [0.]
861 [0.]
862 [0.]
863 [0.]
864 [-9.27380775]
865 [0.]
866 [0.]
867 [0.]
868 [0.]
869 [0.]
870 [0.]
871 [0.]
872 [0.]
873 [0.]
874 [0.]
875 [0.]
876 [0.]
877 [0.]
878 [0.]
879 [0.]
880 [0.]
881 [0.]
882 [0.]
883 [0.]
884 [-2.10384667]
885 [0.]
886 [0.]
887 [0.]
888 [0.]
889 [0.]
890 [0.]
891 [0.]
892 [0.]
893 [0.]
894 [0.]
895 [0.]
896 [0.]
897 [0.]
898 [-3.22671875]
899 [0.]
900 [0.]
901 [0.]
902 [0.]
903 [0.]
904 [0.]
905 [0.]
906 [0.]
907 [0.]
908 [0.]
909 [0.]
910 [0.]
911 [0.]
912 [-25.34718875]
913 [0.]
914 [0.]
915 [0.]
916 [0.]
917 [0.]
918 [0.]
919 [0.]
920 [0.]
921 [0.]
922 [0.]
923 [0.]
924 [0.]
925 [0.]
926 [0.]
927 [0.]
928 [0.]
929 [0.]
930 [0.]
931 [0.]
932 [0.]
933 [0.]
934 [0.]
935 [0.]
936 [0.]
937 [0.]
938 [0.]
939 [0.]
940 [0.]
941 [0.]
942 [7.94924503]
943 [18.40561789]
944 [0.37037857]
945 [0.11659378]
946 [0.]
947 [0.]
948 [0.]
949 [0.]
950 [0.]
951 [0.]
952 [0.]
953 [0.20423813]
954 

1736 [0.]
1737 [0.]
1738 [0.]
1739 [0.]
1740 [0.]
1741 [0.]
1742 [0.]
1743 [0.]
1744 [0.]
1745 [0.]
1746 [0.]
1747 [0.]
1748 [-2.62482271]
1749 [0.]
1750 [0.]
1751 [0.]
1752 [0.]
1753 [0.]
1754 [0.]
1755 [0.]
1756 [0.]
1757 [0.]
1758 [0.]
1759 [0.]
1760 [-42.07868578]
1761 [0.]
1762 [0.]
1763 [0.]
1764 [0.]
1765 [0.]
1766 [0.]
1767 [0.]
1768 [0.]
1769 [0.]
1770 [0.]
1771 [0.]
1772 [0.]
1773 [0.]
1774 [0.]
1775 [0.]
1776 [-2.78713772]
1777 [0.]
1778 [0.]
1779 [0.]
1780 [0.]
1781 [0.]
1782 [0.]
1783 [0.]
1784 [0.]
1785 [0.]
1786 [0.]
1787 [0.]
1788 [0.]
1789 [0.]
1790 [0.]
1791 [0.]
1792 [-57.69891983]
1793 [0.]
1794 [0.]
1795 [0.]
1796 [0.]
1797 [0.]
1798 [0.]
1799 [0.]
1800 [0.]
1801 [0.]
1802 [0.]
1803 [0.]
1804 [0.]
1805 [0.]
1806 [0.]
1807 [0.]
1808 [-23.63796606]
1809 [0.]
1810 [0.]
1811 [0.]
1812 [0.]
1813 [0.]
1814 [0.]
1815 [0.]
1816 [0.]
1817 [0.]
1818 [0.]
1819 [0.]
1820 [0.]
1821 [0.]
1822 [0.]
1823 [0.]
1824 [0.]
1825 [0.]
1826 [0.]
1827 [0.]
1828 [0.]
1829 [0.]
1830 [-7.071

2616 [0.]
2617 [0.]
2618 [0.]
2619 [0.]
2620 [0.]
2621 [0.]
2622 [0.]
2623 [0.]
2624 [-70.88358375]
2625 [0.]
2626 [0.]
2627 [0.]
2628 [0.]
2629 [0.]
2630 [0.]
2631 [0.]
2632 [0.]
2633 [0.]
2634 [0.]
2635 [0.]
2636 [0.]
2637 [0.]
2638 [0.]
2639 [0.]
2640 [-23.767768]
2641 [0.]
2642 [0.]
2643 [0.]
2644 [0.]
2645 [0.]
2646 [0.]
2647 [0.]
2648 [0.]
2649 [0.]
2650 [0.]
2651 [0.]
2652 [0.]
2653 [0.]
2654 [0.]
2655 [0.]
2656 [-60.6038225]
2657 [0.]
2658 [0.]
2659 [0.]
2660 [0.]
2661 [0.]
2662 [0.]
2663 [0.]
2664 [0.]
2665 [0.]
2666 [0.]
2667 [0.]
2668 [0.]
2669 [0.]
2670 [0.]
2671 [0.]
2672 [-85.7213985]
2673 [0.]
2674 [0.]
2675 [0.]
2676 [0.]
2677 [0.]
2678 [0.]
2679 [0.]
2680 [0.]
2681 [0.]
2682 [0.]
2683 [0.]
2684 [0.]
2685 [0.]
2686 [0.]
2687 [0.]
2688 [-65.8822125]
2689 [0.]
2690 [0.]
2691 [0.]
2692 [0.]
2693 [0.]
2694 [0.]
2695 [0.]
2696 [0.]
2697 [0.]
2698 [0.]
2699 [0.]
2700 [0.]
2701 [0.]
2702 [0.]
2703 [0.]
2704 [-76.444512]
2705 [0.]
2706 [0.]
2707 [0.]
2708 [0.]
2709 [0.]
2710 [0

3529 [0.]
3530 [0.]
3531 [0.]
3532 [0.]
3533 [0.]
3534 [0.]
3535 [0.]
3536 [0.]
3537 [0.]
3538 [0.]
3539 [0.]
3540 [0.]
3541 [0.]
3542 [-3.13162844]
3543 [0.]
3544 [0.]
3545 [0.]
3546 [0.]
3547 [0.]
3548 [0.]
3549 [0.]
3550 [0.]
3551 [0.]
3552 [0.95779688]
3553 [0.]
3554 [0.]
3555 [0.]
3556 [0.]
3557 [0.]
3558 [0.]
3559 [0.]
3560 [0.34038506]
3561 [1.37753805]
3562 [2.5559352]
3563 [0.]
3564 [0.]
3565 [0.]
3566 [0.]
3567 [0.]
3568 [0.]
3569 [0.]
3570 [0.]
3571 [0.]
3572 [0.]
3573 [0.]
3574 [0.]
3575 [0.]
3576 [0.]
3577 [0.]
3578 [1.3588974]
3579 [3.2501133]
3580 [4.1321574]
3581 [0.]
3582 [0.]
3583 [0.]
3584 [-78.7553235]
3585 [0.]
3586 [0.]
3587 [0.]
3588 [0.]
3589 [0.]
3590 [0.]
3591 [0.]
3592 [0.]
3593 [0.]
3594 [0.]
3595 [0.]
3596 [0.]
3597 [0.]
3598 [0.]
3599 [0.]
3600 [-18.19594119]
3601 [0.]
3602 [0.]
3603 [0.]
3604 [0.]
3605 [0.]
3606 [0.]
3607 [0.]
3608 [0.]
3609 [0.]
3610 [0.]
3611 [0.]
3612 [0.]
3613 [0.]
3614 [0.]
3615 [0.]
3616 [-55.03599424]
3617 [0.]
3618 [0.]
3619 [0.]


4445 [0.]
4446 [0.]
4447 [0.]
4448 [-64.1704575]
4449 [0.]
4450 [0.]
4451 [0.]
4452 [0.]
4453 [0.]
4454 [0.]
4455 [0.]
4456 [0.]
4457 [0.]
4458 [0.]
4459 [0.]
4460 [0.]
4461 [0.]
4462 [0.]
4463 [0.]
4464 [-68.688227]
4465 [0.]
4466 [0.]
4467 [0.]
4468 [0.]
4469 [0.]
4470 [0.]
4471 [0.]
4472 [0.]
4473 [0.]
4474 [0.]
4475 [0.]
4476 [0.]
4477 [0.]
4478 [0.]
4479 [0.]
4480 [0.]
4481 [0.]
4482 [-14.21552661]
4483 [0.]
4484 [0.]
4485 [0.]
4486 [0.]
4487 [0.]
4488 [0.]
4489 [0.]
4490 [0.]
4491 [0.]
4492 [0.]
4493 [0.]
4494 [0.]
4495 [0.]
4496 [-43.21191855]
4497 [0.]
4498 [0.]
4499 [0.]
4500 [0.]
4501 [0.]
4502 [0.]
4503 [0.]
4504 [0.]
4505 [0.]
4506 [0.]
4507 [0.]
4508 [0.]
4509 [0.]
4510 [0.]
4511 [0.]
4512 [-26.07623461]
4513 [0.]
4514 [0.]
4515 [0.]
4516 [0.]
4517 [0.]
4518 [0.]
4519 [0.]
4520 [0.]
4521 [0.]
4522 [0.]
4523 [0.]
4524 [0.]
4525 [0.]
4526 [0.]
4527 [0.]
4528 [0.]
4529 [0.]
4530 [-7.157307]
4531 [0.]
4532 [0.]
4533 [0.]
4534 [0.]
4535 [0.]
4536 [0.]
4537 [0.]
4538 [0.]
4539 [

5526 [0.]
5527 [0.]
5528 [1.099014]
5529 [0.]
5530 [0.]
5531 [0.]
5532 [0.]
5533 [0.]
5534 [0.]
5535 [0.]
5536 [0.]
5537 [0.]
5538 [-4.60718975]
5539 [0.]
5540 [0.]
5541 [0.]
5542 [0.]
5543 [0.]
5544 [0.]
5545 [0.]
5546 [0.]
5547 [0.]
5548 [0.]
5549 [0.]
5550 [0.]
5551 [0.]
5552 [0.05593334]
5553 [0.]
5554 [0.]
5555 [0.]
5556 [0.]
5557 [0.]
5558 [0.]
5559 [0.]
5560 [0.14148503]
5561 [0.56330032]
5562 [3.54015585]
5563 [6.53784862]
5564 [3.10509864]
5565 [0.]
5566 [0.]
5567 [0.]
5568 [0.]
5569 [0.]
5570 [0.]
5571 [0.]
5572 [0.]
5573 [0.]
5574 [0.]
5575 [0.]
5576 [0.]
5577 [0.]
5578 [2.83487479]
5579 [5.84846745]
5580 [2.42660014]
5581 [0.]
5582 [0.]
5583 [0.]
5584 [0.]
5585 [0.]
5586 [-2.48427025]
5587 [0.]
5588 [0.]
5589 [0.]
5590 [0.]
5591 [0.]
5592 [0.]
5593 [0.]
5594 [1.99292389]
5595 [4.30603425]
5596 [1.9437516]
5597 [0.]
5598 [0.]
5599 [0.]
5600 [0.]
5601 [0.]
5602 [0.]
5603 [0.]
5604 [0.]
5605 [0.]
5606 [0.]
5607 [0.]
5608 [0.13117545]
5609 [0.5179545]
5610 [2.10459533]
5611 [3.